## Preprocessing

In [168]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [169]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns= ['EIN', 'NAME'])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [170]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [171]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [172]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[
    application_df['APPLICATION_TYPE'].value_counts() < cutoff_value
    ].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [173]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [174]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[
    application_df['CLASSIFICATION'].value_counts() >1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[
    application_df['CLASSIFICATION'].value_counts() < cutoff_value
    ].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [175]:
# Calculate the classification counts
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Define your cutoff values
cutoff_1 = 1000
cutoff_2 = 500
cutoff_3 = 5

# Filter to only include classifications with counts less than 1000
classification_counts = classification_counts[classification_counts < cutoff_1]

# Create the bins based on the counts
bin_5 = classification_counts[classification_counts < cutoff_3].index.tolist()   # <5
bin_500 = classification_counts[(classification_counts >= cutoff_3) & (classification_counts < cutoff_2)].index.tolist()  # <500
bin_1000 = classification_counts[classification_counts >= cutoff_2].index.tolist()  # <1000

# Replace the values with bin labels
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(bin_5, '<5')
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(bin_500, '<500')
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(bin_1000, '<1000')

# Check to make sure replacement was successful
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
<500      1439
<1000      777
<5          45
Name: count, dtype: int64


Trying a more radial approach, with much fewer options for classifications. 

In [176]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [177]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df[
    ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']
    ])


In [178]:
# Drop the categorical columns
application_df = application_df.drop(columns= ['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT'])

In [179]:
# Strip whitespace from 'SPECIAL_CONSIDERATIONS'
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].str.strip()

# Map the 'SPECIAL_CONSIDERATIONS' column to numeric binary (1/0 instead of Y/N)
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].map({'Y':1, 'N':0})

Whitespace stripped above, previous attempts at mapping the column resulted in NaN results. There may be unseen whitespace in the column. The `.strip()` fixed this problem.

In [180]:
# Concatenate the dummies with the dataframe
transformed_application_df = pd.concat([application_df, application_dummies], axis=1)

# Convert booleans to integers
transformed_application_df = transformed_application_df.astype(int)

# Display
transformed_application_df

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,108590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,5000,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,0,6692,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,0,142590,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,5000,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,0,5000,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,0,5000,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,0,5000,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [181]:
# Split our preprocessed data into our features and target arrays
X = transformed_application_df.drop('IS_SUCCESSFUL', axis=1)
y = transformed_application_df['IS_SUCCESSFUL']


In [182]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [183]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [184]:
# Find the number of features using shape
number_of_features = X.shape[1]
print(f"# of features: {number_of_features}")

# of features: 44


In [195]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer, make sure to add # of features to `input_dim` parameter
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_62 (Dense)                │ (None, 64)             │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_63 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,633 (22.00 KB)

 Trainable params: 5,633 (22.00 KB)

 Non-trainable params: 0 (0.00 B)

In [196]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [197]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 3s 987us/step - accuracy: 0.7019 - loss: 0.5944
Epoch 2/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step - accuracy: 0.7277 - loss: 0.5550
Epoch 3/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step - accuracy: 0.7303 - loss: 0.5499
Epoch 4/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 930us/step - accuracy: 0.7329 - loss: 0.5473
Epoch 5/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 942us/step - accuracy: 0.7358 - loss: 0.5438
Epoch 6/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - accuracy: 0.7330 - loss: 0.5433
Epoch 7/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - accuracy: 0.7395 - loss: 0.5418
Epoch 8/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 910us/step - accuracy: 0.7310 - loss: 0.5470
Epoch 9/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 961us/step - accuracy: 0.7337 - loss: 0.5458
Epoch 10/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 950us/step - accuracy: 0.7313 - loss: 0.5471
Epoch 11/100
965/965 ━━━━━━━━━━━━━━━━━━━━ 1s 907us/step - accuracy: 0.7337 - loss: 0.5461
Epoch 12/100
965/96

In [198]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

108/108 - 0s - 2ms/step - accuracy: 0.7280 - loss: 0.5638
Loss: 0.5638266205787659, Accuracy: 0.7279883623123169


In [190]:
# Export our model to HDF5 file
nn_model.save('Output/AlphabetSoupCharity_Optimization.h5')